In [ ]:
import pandas as pd
import numpy as np
import altair as alt
import vegafusion

alt.data_transformers.enable("vegafusion") # so that Altair would not complain over too large dataset

# Read and format the data
hw = pd.read_csv('./data/height_weight_gender.csv') 
hw.Height*=100
hw.replace({0: 'Male', 1: 'Female'}, inplace=True)
hw.columns = hw.columns.str.lower()

print(hw.head())
print(hw.shape)


In [ ]:
# Get distribution means and standard deviations for height and weight from the data
mean_h, sigma_h = hw.height.mean(), hw.height.std()
mean_w, sigma_w = hw.weight.mean(), hw.weight.std()

# number of draws
n = 5000

# Generate random samples
h_values = np.random.normal(mean_h, sigma_h, n)
w_values = np.random.normal(mean_w, sigma_w, n)

# Create the simulation dataframe
df_sim = pd.DataFrame({'height': h_values, 'weight': w_values})

alt.Chart(df_sim).mark_circle(size=7, opacity=0.5).encode(
    x=alt.X('weight').scale(domain=[20, 130]),
    y=alt.Y('height').scale(domain=[130,210])
).properties(height=400, width=400)

In [ ]:
# Create a base for plots
base = alt.Chart(hw).properties(
    width=400,
    height=400
) 

# Create scatter plot of weights and heights in the data
points = base.mark_point(filled=True, size=5, opacity=0.3).encode(
    alt.X('weight:Q').scale(domain=[25,125]),
    alt.Y('height:Q').scale(domain=[120,220]),
     tooltip=['height', 'weight']
).properties(height=400, width=400)

# ... and fit linear regression to see the relationship
(points + points.transform_regression('weight', 'height').mark_line(color='orange', size=1.5))

In [ ]:
p = base.mark_bar(opacity=0.75, thickness=100).encode(
    alt.X('height')
        .bin(step=5) 
        .scale(domain=[120, 220])
        .title(None),
    alt.Y('count()')
        .stack(None)
        .title(None)
        .axis(None),
).properties(height=300, width=400).configure(font='SF Compact Display')

p.show()

print(f'std: {hw.height.std():.2f}, mean: {hw.height.mean():.2f}')

In [ ]:

def gaussian_pdf(mean, sigma):

    # Create a range of x values for heights
    x_values = np.linspace(mean - 4*sigma, mean + 4*sigma, 500)

    # Calculate the corresponding y values for the Gaussian PDF
    y_values = (1/(sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x_values - mean)/sigma)**2)

    # Return a dataframe of PDF values
    return pd.DataFrame({
        'x': x_values,
        'y': y_values
    })

mean = 168.6
sigma = 9.8

chart = alt.Chart(gaussian_pdf(mean, sigma)).mark_line(color='red').encode(
    x=alt.X('x', title='x'),
    y=alt.Y('y', title='Probability Density')
    .scale(domain=[0, 0.04])
).properties(
    title=f"Gaussian PDF (μ={mean}, σ={sigma})",
    width=400
)

(p + chart).resolve_scale(y='independent')

In [ ]:
hist_x = base.mark_bar(opacity=0.75, thickness=100, orient='horizontal').encode(
    alt.X('weight')
        .bin(step=5) # step keeps bin size the same
        .scale(domain=[25,125]).axis(None),
    alt.Y('count()')
        .stack(None)
        .axis(None)
).properties(height=70, width=400)

hist_y = base.mark_bar(opacity=0.75, thickness=100).encode(
    alt.Y('height')
        .bin(step=5)
        .scale(domain=[120, 220])
        .axis(None),
    alt.X('count()')
        .stack(None)
        .axis(None),
).properties(width=70, height=400)

(hist_x & (points + points.transform_regression('weight', 'height').mark_line(color='orange', size=1.5) | hist_y)).configure_axis(
        title=None).configure_view(strokeWidth=0) 

In [ ]:
p = base.mark_bar(opacity=0.75, thickness=100, orient='horizontal').encode(
    alt.X('weight')
        .bin(step=5) 
        .scale(domain=[25,125]),
    alt.Y('count()')
        .stack(None)
        .axis(None)
)

p.show()

print(f'std: {hw.weight.std():.2f}, mean: {hw.weight.mean():.2f}')

In [ ]:
mean = hw.weight.mean()
sigma = hw.weight.std()

chart = alt.Chart(gaussian_pdf(mean, sigma)).mark_line(color='red').encode(
    x=alt.X('x', title='weight'),
    y=alt.Y('y', title='Probability Density')
).properties(
    title=f"Gaussian PDF (μ={mean:.2f}, σ={sigma:.2f})"
)

(p + chart).resolve_scale(y='independent')

In [ ]:
print(hw.groupby('gender').weight.mean(),'\n')

print(hw.groupby('gender').weight.std())


In [ ]:
mean_f, mean_m = hw.groupby('gender').weight.mean()
sigma_f, sigma_m = hw.groupby('gender').weight.std()

chart_f = alt.Chart(gaussian_pdf(mean_f, sigma_f)).mark_line(color='red').encode(
    x=alt.X('x').title('weight'),
    y=alt.Y('y').title('stratified pdf')
)

chart_m = alt.Chart(gaussian_pdf(mean_m, sigma_m)).mark_line(color='lightblue').encode(
    x=alt.X('x'),
    y=alt.Y('y').scale(domain=[0, 0.05]).axis(None)
).properties(
    #title=f"Gaussian PDF (μ={mean}, σ={sigma})"
)

((p + chart_f + chart_m).resolve_scale(y='independent'))

In [ ]:
from scipy.stats import norm

# Acquire gender proportions from the data
weights = hw.gender.value_counts(normalize=True)

# Weights for the mixture
w1, w2 = weights.Female, weights.Male  # Get gender weights from the sample

# Generate 500 x values between 30 and 120
x = np.linspace(30, 120, 500)

# Compute PDFs for both Gaussians
pdf1 = norm.pdf(x, mean_f, sigma_f)
pdf2 = norm.pdf(x, mean_m, sigma_m)

# Compute mixture
pdf_mixture = w1 * pdf1 + w2 * pdf2

gmm_df = pd.DataFrame({
    'weight': np.concatenate([x, x, x]),
    'pdf': np.concatenate([pdf1, pdf2, pdf_mixture]),
    'distribution': [f'Gaussian: μ={mean_f:.1f}, σ={sigma_f:.1f}'] * len(x) + [f'Gaussian: μ={mean_m:.1f}, σ={sigma_m:.1f}'] * len(x) + ['Mixture'] * len(x)
})

# Plot both gaussians and mixture
gmm = alt.Chart(gmm_df).mark_line().encode(
    x='weight',
    y='pdf',
    color='distribution:N' 
).properties(
    title="Mixture of Two Gaussians",
    width=400,
    height=400
)

gmm.show()


In [ ]:
gmm = alt.Chart(gmm_df[gmm_df.distribution=='Mixture'], title='Fit with Gaussian mixture').mark_line(color='red').encode(
    x=alt.X('weight:Q').title('weight'),
    y=alt.Y('pdf:Q').scale(domain=[0, 0.025])
)

((p + gmm).resolve_scale(y='independent'))

In [ ]:
selector = alt.selection_point(fields=['gender'])

color_scale = alt.Scale(domain=['Male', 'Female'],
                        range=['teal', 'darkorange']
                        )

base = alt.Chart(hw).properties(height=400, width=400).add_params(selector)

points = base.mark_point(filled=True, size=5, opacity=0.3).encode(
    alt.X('weight:Q').scale(domain=[25,125]),
    alt.Y('height:Q').scale(domain=[120,220]),
    color=alt.condition(
        selector,
        'gender:N',
        alt.value('lightgray'),
        scale=color_scale),
)

hist_x = base.mark_bar(opacity=0.5, thickness=100, orient='horizontal').encode(
    alt.X('weight')
        .bin(step=5) # step keeps bin size the same
        .scale(domain=[25,125]).axis(None),
    alt.Y('count()')
        .stack(None)
        .axis(None),
     alt.Color('gender:N').scale(color_scale).legend(title=None, orient='bottom')
).transform_filter(
    selector
).properties(height=65, width=400)

hist_y = base.mark_bar(opacity=0.5, thickness=100).encode(
    alt.Y('height')
        .bin(step=5) # step keeps bin size the same
        .scale(domain=[120, 220]).axis(None),
    alt.X('count()')
        .stack(None)
        .axis(None),
     alt.Color('gender:N').scale(color_scale)
).transform_filter(
    selector
).properties(width=65, height=400)

(hist_x & (points + points.transform_regression('weight', 'height', groupby=['gender']).mark_line(size=1)| hist_y)).configure_axis(
        title=None).configure_view(strokeWidth=0)